In [19]:
import sys, random, copy

class CAList:
    C = []
    L = []
    S = []
    mthL = ['IO', 'MTF', 'CM', 'EM', 'OL']
    mth = ''  ## IO, MTF, CM, EM
    nAccesses = 0
    searchCount = 0
    duplicateIgnore = True
    debug = False
 
    def error(self, txt):
        print('ERROR: '+str(txt))
        sys.exit()

    def __init__(self, mth='IO', randomElements=0, copyList=None, duplicateIgnore=True, perctotalAccess=2.0, debug=False):
        self.debug = debug
        if (copyList==None):
            self.L = []
        else:
            self.L = copy.deepcopy(copyList)
        self.setMethod(mth)
        self.duplicateIgnore = duplicateIgnore
        if (randomElements>0):
            self.inject(randomElements)
        if debug:
            self.show()
        self.genSearch(perctotalAccess)

    def setMethod(self, mth):
        self.nAccesses = 0
        self.searchCount = 0
        self.C = []
        for i in self.L:
            self.C.append(0)
        if (mth not in self.mthL):
            self.error('method is not one the known methods in '+str(self.mthL))
        self.mth = mth

    def insert(self, value):
        if (not self.duplicateIgnore or value not in self.L):
            if self.mth not in ['OL']:
                self.L.append(value)
                self.C.append(0)
            else:
                if len(self.L) != 0:
                    for i in range(len(self.L)):
                        if self.L[i] > value:
                            self.L.insert(i, value)
                            return
                self.L.append(value)
                    
    def transpose(self, ii):
        if (ii>0):
            T = self.L[ii]
            self.L[ii] = self.L[ii-1]
            self.L[ii-1] = T 
            T = self.C[ii]
            self.C[ii] = self.C[ii-1]
            self.C[ii-1] = T          

    def search(self, value):
        self.searchCount += 1
        if (value not in self.L):
            self.error('value '+str(value)+' is not in the list')
        ii = self.L.index(value)
        nn = 0
        for x in self.L:
            nn += 1
            if (x==value):
                break
        if (self.mth=='MTF'):
            del self.L[ii]
            self.L = [value]+self.L
        if (self.mth=='EM'):
            self.transpose(ii) 
        if (self.mth=='CM'):
            self.C[ii] += 1
            for k in range(ii):
                if (self.C[ii-k-1]<self.C[ii-k]):
                    self.transpose(ii-k)
        self.nAccesses += nn
        return(nn)

    def inject(self, numberElements):
        for i in range(numberElements):
            self.insert(int(100*random.random()))
    
    def show(self):
        print(self.mth+'('+str(self.searchCount)+') -',self.L)
        if (self.mth=='CM'):
            print('Access counts:',self.C)
        print('Cummulative access cost:',self.nAccesses)

    def genSearch(self, perctotalAccess):
        self.S = []
        sz = len(self.L)
        totalAccess = perctotalAccess * sz
        while (totalAccess>0.0):
            self.S.append(self.L[random.randint(0,sz-1)])
            totalAccess -= 1
        print('Generated sequence',self.S)
        
    def searchSeq(self):
        for value in self.S:
            self.search(value)
        if debug:
            self.show()

class SkipList:
    SL = []

    def __init__(self, mth='OL', randomElements=0, copyList=None, duplicateIgnore=True, perctotalAccess=2.0):
        ioL = CAList(mth=mth, randomElements=randomElements, perctotalAccess=perctotalAccess)
        self.SL = [ioL]
        LL = ioL.L
        while len(LL) > 0:
            NN = []
            for value in LL:
                if random.random() < 0.5:
                    NN.append(value)
            if len(LL) > 0:
                self.SL.append(CAList(mth=mth, copyList=NN))
            LL = NN
        level = 0
        for CAL in self.SL:
            print('-- level', level)
            level += 1
            print(CAL.L)


In [20]:
skipL = SkipList(randomElements=10)

Generated sequence [74, 86, 49, 93, 49, 80, 10, 90, 49, 86, 90, 50, 90, 10, 50, 24, 74, 49, 93, 86]
Generated sequence [93, 93, 94, 94, 86, 86, 86, 86]
Generated sequence [49, 49]
Generated sequence []
-- level 0
[10, 24, 49, 50, 74, 80, 86, 90, 93, 94]
-- level 1
[49, 86, 93, 94]
-- level 2
[49]
-- level 3
[]


In [19]:
ioL = CAList(mth='OL',randomElements=2)

OL(0) - [56, 65]
Cummulative access cost: 0
Generated sequence [65, 65, 65, 65]


In [20]:
for mth in ioL.mthL:
    print('--',mth)
    ioL.setMethod(mth)
    ioL.searchSeq()

-- IO
IO(4) - [56, 65]
Cummulative access cost: 8
-- MTF
MTF(4) - [65, 56]
Cummulative access cost: 5
-- CM
CM(4) - [65, 56]
Access counts: [4, 0]
Cummulative access cost: 4
-- EM
EM(4) - [65, 56]
Cummulative access cost: 4
-- OL
OL(4) - [65, 56]
Cummulative access cost: 4
